In [1]:
import numpy as np
import os
import pandas as pd
import re


def list_dir(path):
    # lists all public files in directory (hides files starting with '.')
    files = os.listdir(path)
    return [string for string in files if not string.startswith('.')]


def year2szn(year):
    # changes an integer year into a string for that year's season
    # example: 2022 -> '2021-22'
    return f"{str(year-1)}-{str(year)[-2:]}"


def combine_csvs(data_folder):
    ''' Group all csvs by team
        CSVs included are:
        - szn_totals.csv
        - advanced.csv
        - shooting.csv
        - per100poss.csv
        - standing.csv
        - szn_totals_against.csv'''
    dfs = []
    file_names = ['szn_totals.csv', 'szn_totals_against.csv', 'advanced.csv',
                  'shooting.csv', 'shooting_against.csv',
                  'szn_per100poss.csv', 'szn_per100poss_against.csv', 'standings.csv'
                  ]
    merge_column = 'Team'

    for subfolder in list_dir(data_folder):
        # concat data folder path with subfolder path
        folderpath = os.path.join(data_folder, subfolder)
        if os.path.isdir(folderpath):  # check if directory
            # concat subfolder path with csv path
            csv_path = os.path.join(folderpath, file_names[0])
            # Check if csv file exists in the current subfolder
            if os.path.exists(csv_path):
                # Read the CSV file into a pandas DataFrame, use as base file
                merged_df = pd.read_csv(csv_path)

                # remove any asterisks from team names
                merged_df[merge_column] = [
                    x.strip('*') for x in merged_df[merge_column]]

                # loop through rest of files
                # skips original file, only looks at files in file_names list
                for file_name in file_names[1:]:
                    # Read the current file
                    current_csv_path = os.path.join(folderpath, file_name)
                    if os.path.exists(current_csv_path):
                        # read CSV into pandas DataFrame
                        current_df = pd.read_csv(current_csv_path)
                        # if dataframe has Team as an entry in first row
                        if 'Team' in current_df.iloc[0].values:
                            # then it's a row of column names so assign
                            current_df.columns = current_df.iloc[0]
                            # drop original first row of cols
                            current_df.drop(current_df.index[0], inplace=True)
                        # Merge the current DataFrame with the merged DataFrame
                        if (subfolder == '2022-23') & (file_name == 'standings.csv'):  # weird edge case
                            # only this year each team name for example is Warriors&edgh
                            current_df[merge_column] = [x[:-5]
                                                        for x in current_df[merge_column]]
                        # stripping any asterisks from team names
                        current_df[merge_column] = [
                            x.strip('*') for x in current_df[merge_column]]
                        merged_df = pd.merge(merged_df, current_df, on=merge_column, how='outer', suffixes=(
                            '', '_'+file_name[:-4]))
                # all files from year in one dataframe now

                # add a new column with the year
                merged_df['Year'] = subfolder
                # remove unnamed/fake index column
                merged_df = merged_df.drop(merged_df.columns[0], axis=1)
                
                # Remove NAN columns
                merged_df = merged_df.loc[:, merged_df.columns.notna()]
                #merged_df = merged_df.drop(
                 #   merged_df.columns[merged_df.columns.str.contains('^Unnamed:')], axis=1)
                dupes = np.where([[merged_df.columns.duplicated()]])[2]
                i=1
                while len(dupes)>0:
                    listed = np.array(merged_df.columns)
                    listed[dupes] = merged_df.columns[dupes]+f'_{i}'
                    i+=1
                    merged_df.columns = listed
                    dupes = np.where([[merged_df.columns.duplicated()]])[2]
                merged_df = merged_df.reset_index(drop=True, inplace=False)
                
                
                # Append the modified DataFrame to the list
                dfs.append(merged_df)

    # Concatenate all the DataFrames in the list
    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_df = concatenated_df.sort_values(
        ['Year', 'Rk']).reset_index(drop=True)
    concatenated_df.to_csv('./yearly_stats.csv')
    return concatenated_df

if __name__ == "__main__":
    combine_csvs('./data')
    

In [2]:
all_years = combine_csvs('./data/')

In [3]:
all_years

Rk                   Team     G       MP      FG     FGA    FG%  \
0      1.0         Boston Celtics  69.0      NaN  2065.0  5607.0  0.368   
1      2.0       Rochester Royals  68.0      NaN  2032.0  5377.0  0.378   
2      3.0  Tri-Cities Blackhawks  68.0      NaN  1988.0  6041.0  0.329   
3      4.0     Fort Wayne Pistons  68.0      NaN  2002.0  5927.0  0.338   
4      5.0     Syracuse Nationals  66.0      NaN  1884.0  5365.0  0.351   
...    ...                    ...   ...      ...     ...     ...    ...   
1688  27.0      Charlotte Hornets  82.0  19830.0  3385.0  7413.0  0.457   
1689  28.0        Houston Rockets  82.0  19755.0  3329.0  7287.0  0.457   
1690  29.0        Detroit Pistons  82.0  19805.0  3244.0  7140.0  0.454   
1691  30.0             Miami Heat  82.0  19805.0  3215.0  6991.0  0.460   
1692   NaN         League Average  82.0  19828.0  3442.0  7241.0  0.475   

          3P     3PA    3P%  ...  3P_shooting_against_1_2  \
0        NaN     NaN    NaN  ...                      NaN   
1        NaN     NaN    NaN  ...                      NaN   
2        NaN     NaN    NaN  ...                      NaN   
3        NaN     NaN    NaN  ...                      NaN   
4        NaN     NaN    NaN  ...                      NaN   
...      ...     ...    ...  ...                      ...   
1688   881.0  2669.0  0.330  ...                    0.859   
1689   856.0  2619.0  0.327  ...                    0.817   
1690   934.0  2659.0  0.351  ...                    0.792   
1691   980.0  2852.0  0.344  ...                    0.851   
1692  1012.0  2806.0  0.361  ...                     0.83   

      Unnamed: 23_level_1_shooting_against  %FGA_shooting_against  \
0                                      NaN                    NaN   
1                                      NaN                    NaN   
2                                      NaN                    NaN   
3                                      NaN                    NaN   
4                                      NaN                    NaN   
...                                    ...                    ...   
1688                                   NaN                  0.065   
1689                                   NaN                   0.07   
1690                                   NaN                  0.068   
1691                                   NaN                   0.05   
1692                                   NaN                   0.06   

      Md._shooting_against  Unnamed: 26_level_1_shooting_against  \
0                      NaN                                   NaN   
1                      NaN                                   NaN   
2                      NaN                                   NaN   
3                      NaN                                   NaN   
4                      NaN                                   NaN   
...                    ...                                   ...   
1688                   408                                   NaN   
1689                   435                                   NaN   
1690                   439                                   NaN   
1691                   321                                   NaN   
1692                   389                                   NaN   

      %FGA_shooting_against_1  Md._shooting_against_1  \
0                         NaN                     NaN   
1                         NaN                     NaN   
2                         NaN                     NaN   
3                         NaN                     NaN   
4                         NaN                     NaN   
...                       ...                     ...   
1688                    0.284                    1154   
1689                    0.255                    1114   
1690                    0.281                    1201   
1691                    0.236                     984   
1692                    0.271                    1124   

      Unnamed: 29_level_1_shooting_against 